# integrate.ai API Sample Notebook to run tasks with an AWS task runner.

This is an example notebook that demonstrates creating taskbuilders and running tasks using an AWS task runner. 
For details about required setup and configuration for task runners, see [Using integrate.ai](https://documentation.integrateai.net/#using-integrate-ai).

## Setup
### Set environment variables (or replace inline) with your IAI credentials
Generate and manage this token in the UI, in the Tokens page. 

In [ ]:
from integrate_ai_sdk.api import connect
import os
import json
import pandas as pd

IAI_TOKEN = ""
client = connect(token=IAI_TOKEN)

## Prerequisites

### Download the sample data

You can download sample data from the integrate.ai sample bucket:

For HFL and EDA: [https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip](https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/synthetic.zip)

For PRL and VFL: [https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/vfl.zip](https://s3.ca-central-1.amazonaws.com/public.s3.integrate.ai/integrate_ai_examples/vfl.zip)

### Create a task runner in your workspace

For instructions for how to create an AWS task runner, [see the documentation](https://documentation.integrateai.net/#create-an-aws-task-runner). 

### Upload the sample data to the S3 bucket created for your task runner

**Important: By default the task runner expects your data to be in the bucket that was created when the task runner was provisioned.**

This bucket name takes the form of: `s3://{aws_traskrunner_profile}-{aws_taskrunner_name}.integrate.ai`

For example: `myworkspace-mytaskrunner.integrate.ai`


In [ ]:
aws_taskrunner_profile = "<myworkspace>" # This is your workspace name
aws_taskrunner_name = "<mytaskrunner>" # Task runner name - must match what was supplied in UI to create task runner

base_aws_bucket = f'{aws_taskrunner_profile}-{aws_taskrunner_name}.integrate.ai'

#Where to store the trained model
aws_storage_path = f's3://{base_aws_bucket}/model'

base_aws_bucket #Prints the base_aws_bucket name for reference

### Register the sample datasets

For instructions for how to register datasets, [see the documentation](https://documentation.integrateai.net/#register-a-dataset-aws). 

**In your workspace, register the sample datasets with your task runner with the following names. Replace {base_aws_bucket} with the bucket name for your environment.**

train_one = s3://{base_aws_bucket}/synthetic/train_silo0.parquet

train_two = s3://{base_aws_bucket}/synthetic/train_silo1.parquet

test_parquet = s3://{base_aws_bucket}/synthetic/test.parquet

active_train = s3://{base_aws_bucket}/vfl/active_train.parquet

passive_train = s3://{base_aws_bucket}/vfl/passive_train.parquet

active_test = s3://{base_aws_bucket}/vfl/active_test.parquet

passive_test = s3://{base_aws_bucket}/vfl/passive_test.parquet


**Note:** If you use other datasets or change the names, you **must** update the dataset names in the code example below to run a session succesfully. 

### Set up the taskbuilder 


In [ ]:
from integrate_ai_sdk.taskgroup.taskbuilder.integrate_ai import IntegrateAiTaskBuilder
from integrate_ai_sdk.taskgroup.base import SessionTaskGroup

iai_tb_aws = IntegrateAiTaskBuilder(client=client,
   task_runner_id=aws_taskrunner_name)

# Task 1: Perform EDA in Individual Mode

This example task demonstrates how to run an exploratory data analysis (EDA) session in Individual mode.

In [ ]:
# Specify the dataset configuration
# Note that this example uses registered dataset names 

dataset_config = {"train_one": [], "train_two": []}

In [ ]:
# Create and start the EDA session 

eda_session = client.create_eda_session(
    name="Testing notebook - EDA",
    description="I am testing EDA session creation with a task runner through a notebook",
    data_config=dataset_config,
).start()

eda_session.id  #Prints the EDA session ID for reference

In [ ]:
# Create a task group with one task for each of the clients joining the session

eda_task_group_context = (
        SessionTaskGroup(eda_session) \
        .add_task(iai_tb_aws.eda(dataset_name="train_one"))\
        .add_task(iai_tb_aws.eda(dataset_name="train_two"))\
        .start()
)

In [ ]:
# Check the task group status

for i in eda_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

eda_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

eda_task_group_context.wait(60*5, 2)

### Task 1: EDA session complete

In [ ]:
# Session complete, retrieve the results 

results = eda_session.results()["train_one"]
results.describe()

In [ ]:
results.mean().shape

In [ ]:
dataset_one = eda_session.results()["train_one"]
dataset_one_count = dataset_one["x0"].count()
dataset_one["x0"].mean()

# Task 2: Create a PRL Session for linking two or more datasets

To create a PRL session, specify a `dataset_config` dictionary indicating the client names and columns to use as identifiers to link the datasets to each other. The number of expected clients will be inferred as the number of items in dataset_config (i.e., two). These client names are referenced for the compute on the PRL session and for any sessions that use the PRL session downstream.

For this session, two clients are going to be providing data. Client 1 and client 2 are naming their clients active_client and passive_client respectively. Their datasets will be linked by the "id" column in any provided datasets.

Detailed information about PRL is available in the [documentation](https://documentation.integrateai.net/#private-record-linkage-prl-sessions).

In [ ]:
# Specify PRL dataset configuration 

prl_data_config = {
  "clients": {
    "active_client": {
      "id_columns": [
        "id"
      ],
    },
    "passive_client": {
      "id_columns": [
        "id"
      ],
    }
  }
}


In [ ]:
# Create and start PRL session

prl_session = client.create_prl_session(
    name="Testing notebook - PRL",
    description="I am testing PRL session creation with a task runner through a notebook",
    data_config=prl_data_config
).start()

prl_session.id #Prints the session ID for reference

In [ ]:
# Create a task group with one task for each of the clients joining the session

task_group = (SessionTaskGroup(prl_session)\
    .add_task(iai_tb_aws.prl(train_dataset_name="active_train", test_dataset_name="active_test", client_name="active_client"))\
    .add_task(iai_tb_aws.prl(train_dataset_name="passive_train", test_dataset_name="passive_test", client_name="passive_client"))
)

task_group_context = task_group.start()

In [ ]:
#Check the status of the task group

for i in task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

task_group_context.wait(60*5, 2)

### Task 2: PRL Session Complete!
Now you can view the overlap stats for the datasets.

In [ ]:
# View PRL session metrics

metrics = prl_session.metrics().as_dict()
metrics

## Task 3: Create a VFL Training Session using the PRL session from Task 2

To create a VFL train session, specify the `prl_session_id` indicating the session you just ran to link the datasets together. The `vfl_mode` needs to be set to `train`.

For more information about vertical federated learning with a SplitNN model strategy, see the [documentation](https://documentation.integrateai.net/#vfl-splitnn-model-training). 

In [ ]:
# Specify the model and data configurations

model_config = {
    "strategy": {"name": "SplitNN", "params": {}},
    "model": {
        "feature_models": {
            "passive_client": {"params": {"input_size": 7, "hidden_layer_sizes": [6], "output_size": 5}},
            "active_client": {"params": {"input_size": 8, "hidden_layer_sizes": [6], "output_size": 5}},
        },
        "label_model": {"params": {"hidden_layer_sizes": [5], "output_size": 2}},
    },
    "ml_task": {
        "type": "classification",
        "params": {
            "loss_weights": None,
        },
    },
    "optimizer": {"name": "SGD", "params": {"learning_rate": 0.2, "momentum": 0.0}},
    "seed": 23,  # for reproducibility
}

data_config = {
        "passive_client": {
            "label_client": False,
            "predictors": ["x1", "x3", "x5", "x7", "x9", "x11", "x13"],
            "target": None,
        },
        "active_client": {
            "label_client": True,
            "predictors": ["x0", "x2", "x4", "x6", "x8", "x10", "x12", "x14"],
            "target": "y",
        },
    }

In [ ]:
# Create and start a VFL training session

vfl_train_session = client.create_vfl_session(
    name="Testing notebook - VFL Train",
    description="I am testing VFL Train session creation with a task runner through a notebook",
    prl_session_id=prl_session.id,
    vfl_mode='train',
    min_num_clients=2,
    num_rounds=2,
    package_name="iai_ffnet",       # this session is using a SplitNN method
    data_config=data_config,
    model_config=model_config
).start()

vfl_train_session.id    #Prints the session ID for reference

In [ ]:
# Create and start a task group with one task for each of the clients joining the session
# This example uses registered dataset names.

storage_path = f"{aws_storage_path}/vfl/{vfl_train_session.id}"

vfl_task_group_context = (SessionTaskGroup(vfl_train_session)\
    .add_task(iai_tb_aws.vfl_train(train_dataset_name="active_train", test_dataset_name="active_test", 
                                    batch_size=1024,  
                                    client_name="active_client", 
                                    storage_path = storage_path))\
    .add_task(iai_tb_aws.vfl_train(train_dataset_name="passive_train", test_dataset_name="passive_test", 
                                    batch_size=1024, 
                                    client_name="passive_client", 
                                    storage_path = storage_path))\
    .start())


In [ ]:
# Check the status of the tasks

for i in vfl_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

vfl_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

vfl_task_group_context.wait(60*5, 2)

### Task 3: VFL Session Complete!
Now you can view the VFL training metrics and start making predictions

In [ ]:
metrics = vfl_train_session.metrics().as_dict()
metrics

In [ ]:
fig = vfl_train_session.metrics().plot()

## Task 4: Make a Prediction on the trained VFL model

To create a VFL predict session, specify the `prl_session_id` indicating the session you ran to link the datasets together. You also need the `training_id` of the VFL train session that was run using the same `prl_session_id`. 

The `vfl_mode` must be set to `predict`.

For more information, see the [documentation](https://documentation.integrateai.net/#vfl-prediction-session-example).

In [ ]:
# Create and start a VFL predict session

vfl_predict_session = client.create_vfl_session(
    name="Testing notebook - VFL Predict",
    description="I am testing VFL Predict session creation with an AWS task runner through a notebook",
    prl_session_id=prl_session.id,
    training_session_id=vfl_train_session.id,
    vfl_mode="predict",
    data_config=data_config
).start()

vfl_predict_session.id  # Prints the session ID for reference

In [ ]:
# Create and start a task group with one task for the server, and one for each of the clients joining the session
# This example uses registered dataset names

storage_path = f"{vfl_predict_active_storage_path}_{vfl_predict_session.id}.csv"

vfl_predict_task_group_context = (SessionTaskGroup(vfl_predict_session)\
.add_task(iai_tb_aws.vfl_predict(
        client_name="active_client", 
        dataset_name="active_test",
        raw_output=True,
        batch_size=1024, 
        storage_path = storage_path))\
.add_task(iai_tb_aws.vfl_predict(
        client_name="passive_client",
        dataset_name="passive_test",
        batch_size=1024,
        raw_output=True,
        storage_path = storage_path))\
.start())

In [ ]:
# Check the status of the tasks

for i in vfl_predict_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

vfl_predict_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

vfl_predict_task_group_context.wait(60*5, 2)

### VFL Predict Session Complete!

Now you can view the VFL predictions and evaluate the performance as needed

In [ ]:
# Retrieve the metrics

metrics = vfl_predict_session.metrics().as_dict()
metrics

In [ ]:
presigned_result_urls = vfl_predict_session.prediction_result()

print(vfl_predict_active_storage_path)
df_pred = pd.read_csv(presigned_result_urls.get(storage_path))

df_pred.head()